In [247]:
%%capture

# installing necessary pdf conversion packages via pip
# the '%%capture' at the top of this cell suppresses the output (which is normally quite long and annoying looking). 
# You can remove or comment it out if you prefer to see the output. 
!pip install nltk
!pip install autocorrect        
!pip install pyspellchecker 
!pip install spacy -q
!python -m spacy download en_core_web_lg -q


In [249]:
import os                         # os is a module for navigating your machine (e.g., file directories).

import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions
from nltk import sent_tokenize  
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import webtext

import pandas as pd
pd.set_option('display.max_colwidth', 200)
import numpy as np
import statistics
import datetime
date = datetime.date.today()

from autocorrect import Speller   # things we need for spell checking
check = Speller(lang='en')
import codecs
import csv                        # csv is for importing and working with csv files

from collections import Counter

import statistics
import re                         # things we need for RegEx corrections

import string 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000 #or any large value, as long as you don't run out of RAM

import math 

English_punctuation = "-!\"#$%&()'*-–+,./:;<=>?@[\]^_`{|}~''“”"      # Things for removing punctuation, stopwords and empty strings
table_punctuation = str.maketrans('','', English_punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\mzyssjkc\

In [98]:
os.listdir("..\\results")  

['ESHG2001abstractICHG.csv', 'ESHG2002Abstracts.csv', 'ESHG2016Abstracts.csv']

In [91]:
df = pd.read_csv(r'..\results\ESHG2001AbstractICHG.csv')
df.head()


,Title,Session_Code,Text,Email,Authors_and_Affiliations,Year
0,Progress in sequencing and annotating the huma...,PS01.,Despite the availability of most of the human ...,jsbach@genoscope.cns.fr,Weissenbach Genoscope Evry F,2001
1,Protein Structures Inherited Mutations and Di...,PS02.,The mechanisms whereby inherited DNA mutations...,thornton@biochem.ucl.ac.uk,M. ThorntonUniversity College Biochemistry a...,2001
2,Mutations and Their Effect on Protein Structure,PS03.,Missense mutations can knock out residues in p...,hjt1001@cam.ac.uk,R. FershtCentre for Protein Engineering Unive...,2001
3,Pharmacogenetics A Disruptive Technology For ...,PS04.,The Genome Project has accelerated the availab...,mdb4138@glaxowellcome.co.uk,Roses Senior Vice President Genetics Researc...,2001
4,Predictive Testing for Huntington Disease 15...,PS05.,Predictive genetic testing (PT) for Huntington...,mrh@cmmt.ubc.ca,R. Hayden Centre for Molecular medicine & The...,2001


In [166]:
files = []
def import_results(input):
    for f in os.listdir(input):
        f = pd.read_csv(input + '\\'+ f)
        files.append(f)
    output = pd.concat(files)
    return output


In [212]:
all_results = import_results("..\\results")
len(all_results)


69204

In [211]:
no_null_texts = all_results[~all_results['Text'].isnull()]
len(no_null_texts)

61644

In [208]:
matched_texts = no_null_texts[~no_null_texts['Text'].str.contains('autis|Autis|ASD|Asperger|asperger')]
len(matched_texts)

54570

In [268]:
isolated_sent = matched_texts.explode('tokenized_sents')
len(isolated_sent)

581070

In [271]:
matched_sents = isolated_sent[~isolated_sent['tokenized_sents'].str.contains('autis|Autis|ASD|Asperger|asperger')]
len(matched_sents)

581070

In [272]:
matched_sents

,Session_Code,Text,Email,Authors_and_Affiliations,Year,Author,Affiliations,tokenized_sents
Title,,,,,,,,
Progress in sequencing and annotating the human genome,PS01.,Despite the availability of most of the human genome sequence the accu rate identification of genes on the DNA sequence remains to be continu ously improved and updated. This procedure relies on ...,jsbach@genoscope.cns.fr,Weissenbach Genoscope Evry F,2001,NaN,NaN,Despite the availability of most of the human genome sequence the accu rate identification of genes on the DNA sequence remains to be continu ously improved and updated.
Progress in sequencing and annotating the human genome,PS01.,Despite the availability of most of the human genome sequence the accu rate identification of genes on the DNA sequence remains to be continu ously improved and updated. This procedure relies on ...,jsbach@genoscope.cns.fr,Weissenbach Genoscope Evry F,2001,NaN,NaN,This procedure relies on the combined useof a variety of tools including similarity searches and exon/gene predictionprograms.
Progress in sequencing and annotating the human genome,PS01.,Despite the availability of most of the human genome sequence the accu rate identification of genes on the DNA sequence remains to be continu ously improved and updated. This procedure relies on ...,jsbach@genoscope.cns.fr,Weissenbach Genoscope Evry F,2001,NaN,NaN,Similarity searches rely on a set of sequence data that has been increas ing very substantially and rapidly during the past months.
Progress in sequencing and annotating the human genome,PS01.,Despite the availability of most of the human genome sequence the accu rate identification of genes on the DNA sequence remains to be continu ously improved and updated. This procedure relies on ...,jsbach@genoscope.cns.fr,Weissenbach Genoscope Evry F,2001,NaN,NaN,These includehuman cDNAs mouse genomic DNA and cDNAs and puffer fish genomicsequences.
Progress in sequencing and annotating the human genome,PS01.,Despite the availability of most of the human genome sequence the accu rate identification of genes on the DNA sequence remains to be continu ously improved and updated. This procedure relies on ...,jsbach@genoscope.cns.fr,Weissenbach Genoscope Evry F,2001,NaN,NaN,With these resources the definition of a vertebrate set ofgenes is progressively approaching completion.The use of a compact vertebrate genome (the pufferfish Fugu rubripesand Tetraodon nigrovir...
...,...,...,...,...,...,...,...,...
Response to Positive Cancer Genetic Testing Results The Cyprus Experience,EMP1.09,"Social parameters, family dynamics and the general appreciation for genetic testing by wider society vary from one culture to the next. It is therefore important to consider these factors within ...",NaN,NaN,2016,"M. Loizidou, K. Kyriacou, V. Christophidou Anastasiadou,","Clinical Genetics Clinic, The Cyprus Institute of Neurology and Genetics, Nicosia, Cyprus, The Cyprus Institute of Neurology and Genetics, Nicosia, Cyprus, Department of Electron Microscopy, The...","Social parameters, family dynamics and the general appreciation for genetic testing by wider society vary from one culture to the next."
Response to Positive Cancer Genetic Testing Results The Cyprus Experience,EMP1.09,"Social parameters, family dynamics and the general appreciation for genetic testing by wider society vary from one culture to the next. It is therefore important to consider these factors within ...",NaN,NaN,2016,"M. Loizidou, K. Kyriacou, V. Christophidou Anastasiadou,","Clinical Genetics Clinic, The Cyprus Institute of Neurology and Genetics, Nicosia, Cyprus, The Cyprus Institute of Neurology and Genetics, Nicosia, Cyprus, Department of Electron Microscopy, The...","It is therefore important to consider these factors within a specific culture when hoping to maximize the effective awareness of the appropriate individuals of this culture, regarding their geneti..."
Response to Positive Cancer Genetic Testing Results The Cyprus Ex

In [ ]:
search for word count - mutation - show frequency decreasing over time, increase in variant? gene change? 

disease also to decrease and condition or disorder to increase

In [209]:
#for saving output
os.makedirs('folder/subfolder', exist_ok=True)  
df.to_csv('folder/subfolder/out.csv') 